In [1]:
import numpy as np
import pandas as pd
import os
from sqlalchemy import create_engine
import seaborn as sns
import pytz
import datetime
from bs4 import BeautifulSoup

In [2]:
pw = os.getenv('mysql')
connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/'
engine = create_engine(connection_string)

In [3]:
def remove_html(text):
    soup = BeautifulSoup(text,'html.parser')
    return soup.get_text()

### Data cleaning and preparation of Course List

In [4]:
df_a = pd.read_sql("SELECT * FROM udemy.df_courses_a", engine)
df_b = pd.read_sql("SELECT * FROM udemy.df_courses_b", engine)
df_c = pd.read_sql("SELECT * FROM udemy.df_courses_c", engine)
df_d = pd.read_sql("SELECT * FROM udemy.df_courses_d", engine)
df_music = pd.read_sql("SELECT * FROM udemy.df_music", engine)

In [5]:
# Concatenate all the dfs

In [6]:
course_list = pd.concat([df_a,df_b,df_c,df_d,df_music])

In [7]:
course_list.shape

(52214, 23)

In [8]:
# Drop duplicates

In [9]:
course_list = course_list.drop_duplicates(subset='course_id')

In [10]:
course_list['course_id'].duplicated().sum()

0

In [11]:
# drop the Discount and Discount Price columns

In [12]:
course_list = course_list.drop(['discount','discount_price'], axis=1)

In [13]:
# drop null values

In [14]:
course_list.isna().sum()

course_id                  0
title                      0
published_time             0
num_subscribers            0
price                      0
avg_rating                 0
avg_recent_rating          0
num_reviews                0
num_lectures               0
num_quizzes                0
is_practice_test_course    0
language                   0
content_length             0
primary_category           0
primary_subcategory        0
level                      0
instructor                 0
instructor_job_title       5
headline                   0
description                0
url                        0
dtype: int64

In [15]:
course_list = course_list.dropna()

In [16]:
course_list.isna().sum().sum()

0

In [17]:
course_list.shape

(39512, 21)

In [18]:
# Check Data Types

In [19]:
course_list.dtypes # Need to correct published_time,price,is_practice_test_course

course_id                    int64
title                       object
published_time              object
num_subscribers              int64
price                       object
avg_rating                 float64
avg_recent_rating          float64
num_reviews                  int64
num_lectures                 int64
num_quizzes                  int64
is_practice_test_course      int64
language                    object
content_length               int64
primary_category            object
primary_subcategory         object
level                       object
instructor                  object
instructor_job_title        object
headline                    object
description                 object
url                         object
dtype: object

In [20]:
# correct published_time column

In [21]:
course_list['published_time'] = pd.to_datetime(course_list['published_time'])

In [22]:
course_list['published_time'].dtypes

datetime64[ns, UTC]

In [23]:
# correct price column

In [24]:
course_list['price'] = course_list['price'].str.replace('€','').replace('Free',0.00).astype(float)

In [25]:
# correct is_practice_test_course column

In [26]:
course_list['is_practice_test_course'] = course_list['is_practice_test_course'].astype(bool)

In [27]:
# add months_since_created column

In [28]:
time_zone = pytz.timezone('UTC')

In [29]:
current_time = datetime.datetime.now(time_zone)

In [30]:
course_list['months_since_created'] = round((current_time - course_list['published_time']).dt.days/30, 2)

In [31]:
course_list['published_time'] = course_list['published_time'].dt.date

In [32]:
# create num_subscribers_monthly column

In [33]:
course_list['num_subscribers_monthly'] = round(course_list['num_subscribers']/course_list['months_since_created'])

In [34]:
# Clean categorical data in Description column

In [35]:
course_list['description'] = course_list['description'].apply(remove_html)

In [36]:
# Fill blank value in 'level' column

In [37]:
course_list['level'] = course_list['level'].replace('','All Levels')

### Data cleaning and preparation of Course Reviews

In [38]:
reviews_group1_a = pd.read_sql("SELECT * FROM udemy.df_reviews_top_a", engine)
reviews_group1_b = pd.read_sql("SELECT * FROM udemy.df_reviews_top_b", engine)
reviews_group2 = pd.read_sql("SELECT * FROM udemy.df_reviews_medium", engine)

In [39]:
reviews_group1 = pd.concat([reviews_group1_a, reviews_group1_b])

In [40]:
# Check duplicates

In [41]:
reviews_group1.duplicated().sum()

0

In [42]:
reviews_group2.duplicated().sum()

0

In [43]:
# check null values

In [44]:
reviews_group1.isna().sum().sum()

0

In [45]:
reviews_group2.isna().sum().sum()

0

In [46]:
# check datatypes

In [47]:
reviews_group1.dtypes # need to correct created_time column

review_id         int64
rating          float64
comment          object
created_time     object
user             object
course_id         int64
dtype: object

In [48]:
reviews_group2.dtypes

review_id         int64
rating          float64
comment          object
created_time     object
user             object
course_id         int64
dtype: object

In [49]:
reviews_group1['created_time'] = pd.to_datetime(reviews_group1['created_time'],utc=True).dt.date

In [50]:
reviews_group2['created_time'] = pd.to_datetime(reviews_group2['created_time'],utc=True).dt.date

### Data cleaning and preparation of Course Curricula

In [51]:
curricula_group1 = pd.read_sql("SELECT * FROM udemy.df_curricula_top", engine)
curricula_group2 = pd.read_sql("SELECT * FROM udemy.df_curricula_medium", engine)

In [52]:
# check duplicates

In [53]:
curricula_group1.duplicated().sum()

0

In [54]:
curricula_group2.duplicated().sum()

0

In [55]:
# check null values

In [56]:
curricula_group1.isna().sum().sum()

0

In [57]:
curricula_group2.isna().sum().sum()

0

In [58]:
# check data types (no changes needed)

In [59]:
curricula_group1.dtypes

type_session_1     object
title_session_1    object
type_session_2     object
title_session_2    object
type_session_3     object
title_session_3    object
type_session_4     object
title_session_4    object
course_id           int64
dtype: object

In [60]:
curricula_group2.dtypes

type_session_1     object
title_session_1    object
type_session_2     object
title_session_2    object
type_session_3     object
title_session_3    object
type_session_4     object
title_session_4    object
course_id           int64
dtype: object

### Creating the instructor table

In [61]:
course_list.shape

(39512, 23)

In [62]:
new1 = course_list.drop_duplicates(subset=['instructor','instructor_job_title'],keep='first').reset_index(drop=True).reset_index().rename(columns={'index':'instructor_id'})

In [63]:
new1['instructor_id'] = new1['instructor_id']+1

In [64]:
instructors = new1[['instructor_id','instructor','instructor_job_title']]
instructors

,instructor_id,instructor,instructor_job_title
0,1,Divine YJ Truth,I am a Producer
1,2,Mahmoud Elhalabi,Doctorate of business adminstration - Training...
2,3,Azharul Rafy,Content Creator & Digital Marketer
3,4,Enes Karakaş,Developer & @shareCodeCamp
4,5,Youssef Zidan,Graphic Designer
...,...,...,...
19583,19584,Hans Weemaes | LEED AP | WELL AP | TRUE | FITWELL,Sustainability Professional with over 15 years...
19584,19585,Sarah Laws,"Entrepreneur, Marketing Nerd, and Martini Lover"
19585,19586,Sagar Chakraborty,Business Coach
19586,19587,Giant Self,Solutions For Smart Business Owners


In [65]:
course_list = course_list.merge(instructors, how='inner', on = ['instructor','instructor_job_title'],copy=False)

In [66]:
course_list = course_list.drop(['instructor','instructor_job_title'], axis=1)

### Send cleaned data to SQL

In [67]:
# Concatenating the two groups

In [68]:
reviews = pd.concat([reviews_group1,reviews_group2])
curricula = pd.concat([curricula_group1, curricula_group2])

In [69]:
course_list.to_sql("course_list", connection_string, schema='udemy_final', if_exists='replace', index = False)
reviews.to_sql("reviews", connection_string, schema='udemy_final', if_exists='replace', index = False)
curricula.to_sql("curricula", connection_string, schema='udemy_final', if_exists='replace', index = False)
instructors.to_sql("instructors", connection_string, schema='udemy_final', if_exists='replace', index = False)

19588